In [7]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib widget
plt.close('all')
import numpy as np
from random import choice
import os
from scipy.signal import savgol_filter
path_save = r"C:\Users\coren\Dropbox\Apps\Snellius2\data_tables\time_plate_infos.pick"
time_plate_info = pd.read_pickle(path_save)
path_save = r"C:\Users\coren\Dropbox\Apps\Snellius2\data_tables\global_hypha_infos.pick"
global_hypha_info = pd.read_pickle(path_save)
path_save = r"C:\Users\coren\Dropbox\Apps\Snellius2\data_tables\time_hypha_infos.pick"
time_hypha_info = pd.read_pickle(path_save)
plt.style.use('presentation.mplstyle')

In [11]:
def get_branch(row):
    if row['Plate']==94:
        hyph = row['end']
        name = f'branches/ts_{hyph}_auto.npy'
        if not os.path.isfile(name):
            return(None)
        else:
            t_real = np.load(name)
            return(row['time_since_emergence'] in t_real)

In [12]:
global_hypha_info['lapse']=global_hypha_info['time_stop_growth']-global_hypha_info['time_init_growth']
time_hypha_info['distance_final_pos_smooth'] = time_hypha_info['distance_final_pos'].rolling(window = 5).mean()
time_hypha_info['new_branch'] = time_hypha_info.apply(get_branch, axis=1)

In [26]:
plates = [792,799,800,91]
select_time_plate = time_plate_info.loc[time_plate_info['Plate'].isin(plates)]
select_time = time_hypha_info.loc[time_hypha_info['Plate'].isin(plates)]
select_global = global_hypha_info.loc[global_hypha_info['Plate'].isin(plates)]
select_global = select_global.loc[select_global['lapse']>0]
hyphae_interest= select_global['hypha'].unique()
# hyphae_select= [hyph for hyph in hyphae_interest if len(select_time.loc[(select_time['end']==hyph)&(select_time['has_reached_final_pos']=='False')])]
select_time = select_time.loc[select_time['end'].isin(hyphae_interest)]
# select_time = select_time.loc[select_time['has_reached_final_pos']=='False']
select_time = select_time.loc[select_time['speed'].between(50,350)]
select_time['speed'] = select_time['speed'].astype(float)


select_time = select_time.loc[select_time['distance_final_pos']>200]
select_time["int_time"] = select_time['time_since_emergence'].astype(int)
select_time = select_time.loc[select_time['in_ROI']=='True']

In [33]:
select_time.hist('speed',bins=30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<AxesSubplot:title={'center':'speed'}>]], dtype=object)

In [126]:
plates = [94,152,792,799,91]
select_time_plate = time_plate_info.loc[time_plate_info['Plate'].isin(plates)]
select_time = time_hypha_info.loc[time_hypha_info['Plate'].isin(plates)]
select_global = global_hypha_info.loc[global_hypha_info['Plate'].isin(plates)]
select_global = select_global.loc[select_global['lapse']>5]
hyphae_interest= select_global['hypha'].unique()
# hyphae_select= [hyph for hyph in hyphae_interest if len(select_time.loc[(select_time['end']==hyph)&(select_time['has_reached_final_pos']=='False')])]
select_time = select_time.loc[select_time['end'].isin(hyphae_interest)]
# select_time = select_time.loc[select_time['has_reached_final_pos']=='False']
select_time = select_time.loc[select_time['speed'].between(50,350)]
select_time['speed'] = select_time['speed'].astype(float)


select_time = select_time.loc[select_time['distance_final_pos']>400]
select_time["int_time"] = select_time['time_since_emergence'].astype(int)
select_time = select_time.loc[select_time['in_ROI']=='True']

In [105]:
plate = 94

In [127]:
fig, ax = plt.subplots()

ax.plot(select_time.groupby('int_time')['speed'].mean())
ax.set_xlim((0,20))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 20.0)

In [116]:
fig, ax = plt.subplots()

select_time.plot('time_since_emergence','speed',kind = 'scatter',alpha = 0.05,s = 80,ax = ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='time_since_emergence', ylabel='speed'>

In [68]:
hyphae = select_time['end'].unique()
tot = 1
hyph_plot = [choice(hyphae) for i in range(tot)]

In [128]:
to_plot

,end,distance_final_pos,timedelta,time_since_emergence,speed,timestep,timestep_init,time_init,degree,width_tip_edge,...,Plate,folder,t,density_window500,density_window1000,density_window2000,distance_final_pos_smooth,new_branch,int_time,speed_smooth
18054,18054,5932.865244,1.0,0.0,258.807562,43,43,43.15,1,8.136291,...,94,Analysis_1636046622780809853_0_100_Version9,43,2840.57971,NaN,NaN,1234.861573,None,0,NaN
18054,18054,5792.530104,1.0,1.0,183.631298,44,43,43.15,1,8.609836,...,94,Analysis_1636046622780809853_0_100_Version9,44,2900.869565,NaN,NaN,1188.137912,None,1,221.219430
18054,18054,5686.607864,1.0,2.0,162.433766,45,43,43.15,1,8.682118,...,94,Analysis_1636046622780809853_0_100_Version9,45,2680.57971,NaN,NaN,1159.153858,None,2,173.032532
18054,18054,5593.313866,1.0,3.0,195.236549,46,43,43.15,1,8.434829,...,94,Analysis_1636046622780809853_0_100_Version9,46,3042.318841,NaN,NaN,1129.899143,None,3,178.835158
18054,18054,5484.499066,1.0,4.0,175.090296,47,43,43.15,1,8.410733,...,94,Analysis_1636046622780809853_0_100_Version9,47,3056.231884,NaN,NaN,1107.164685,None,4,185.163423
18054,18054,5382.789704,1.0,5.0,175.023915,48,43,43.15,1,8.436991,...,94,Analysis_1636046622780809853_0_100_Version9,48,2529.855072,NaN,NaN,1086.233085,None,5,175.057106
18054,18054,5290.57804,1.0,6.0,199.252452,49,43,43.15,1,8.418413,...,94,Analysis_1636046622780809853_0_100_Version9,49,1509.565217,NaN,NaN,1066.608245,None,6,187.138184
18054,18054,5180.508566,1.0,7.0,183.956623,50,43,43.15,1,8.465239,...,94,Analysis_1636046622780809853_0_100_Version9,50,1194.202899,NaN,NaN,1045.192500,None,7,191.604538
18054,18054,5081.170436,1.0,8.0,211.479901,51,43,43.15,1,8.416072,...,94,Analysis_1636046622780809853_0_100_Version9,51,1212.753623,NaN,NaN,1022.899516,None,8,197.718262
18054,18054,4961.064402,1.0,9.0,54.688584,52,43,43.15,1,8.513,...,94,Analysis_1636046622780809853_0_100_Version9,52,1196.521739,NaN,NaN,998.599995,None,9,133.084243


In [120]:
plt.close('all')
# hyph = hyphae[i]
i =0
# hyph_plot = [choice(hyphae) for i in range(tot)]
hyph = hyph_plot[0]
# hyph = 5435
to_plot = select_time.loc[select_time['end']==hyph]
to_plot['speed_smooth'] = to_plot['speed'].rolling(window = 2).mean()
to_plot.plot('time_since_emergence','speed',xlabel = 'Time (h)',ylabel = 'speed (um.h-1)',title = f'hyph {hyph}, plate {plate}',)
# plt.vlines(t_real,np.min(to_plot['speed_smooth']),np.max(to_plot['speed_smooth']),color="red",alpha = 0.3)
i+=1

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …